# Đánh giá Faithfulness (DeltaF1) cho Influence Function truyền thống (Baseline 1) trên các bộ dữ liệu Test (ParaVE, GramVar).



## Mục đích

- Đánh giá **mức độ trung thành (faithfulness)** của **điểm ảnh hưởng truyền thống** (traditional influence score) bằng thước đo **DeltaF1**.
- Mỗi cặp (test sample, train neighbor) được kiểm tra như sau:
  - Tính **F1 gốc** của mô hình fine-tuned hiện tại trên mẫu test.
  - Thực hiện **một bước cập nhật gradient** (SGD) trên mô hình với một mẫu train (neighbor).
  - Tính lại **F1 mới** trên mẫu test với mô hình đã cập nhật.
  - Tính **DeltaF1 = F1_new − F1_original** và lưu kèm với neighbor.
- Kết quả được dùng để:
  - Phân tích mối tương quan giữa **influence score truyền thống** và **DeltaF1**.
  - So sánh với các phương pháp IF được đề xuất khác (như IF* trên span vector).

## Đầu vào

### 1. Cấu hình và tham số

- Thiết bị:
  - `device = torch.device("cuda" if torch.cuda.is_available() else "cpu")`
- Tham số thí nghiệm:
  - `ETA_LEARNING_RATE = 1e-1`  
    Hệ số học (learning rate) cho một bước SGD.
  - `K_NEIGHBORS_TO_EVALUATE = 1`  
    Chỉ đánh giá **láng giềng gần nhất** cho mỗi mẫu test (mặc dù comment ghi 3, nhưng giá trị thực là 1).

### 2. Đường dẫn

- Gốc:
  - `drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'`
- Thư mục kết quả:
  - `output_dir = os.path.join(drive_base_path, 'matching_results')`
- Mô hình fine-tuned:
  - `final_model_path = 'Finetuned_Models/biobert-srl-best-model'`

### 3. Danh sách bộ dữ liệu baseline cần tính Faithfulness

Biến `datasets_to_run_faithfulness`:

1. ParaVE Test (Baseline 1)
   - File Influence input:
     - `influence_results_baseline1_traditional_parave_test.json`
     - Chứa:
       - `test_sample`
       - `neighbors` với `influence_score_traditional` (điểm ảnh hưởng truyền thống)
   - File Faithfulness output:
     - `faithfulness_scores_baseline1_traditional_parave_test.json`
     - Sẽ được tạo mới, có thêm trường `DeltaF1` trong từng neighbor.

2. GramVar Test (Baseline 1)
   - File Influence input:
     - `influence_results_baseline1_traditional_gramvar_test.json`
   - File Faithfulness output:
     - `faithfulness_scores_baseline1_traditional_gramvar_test.json`

### 4. Dữ liệu gốc (Train / Test)

- Train:
  - `train_data_dir_gramvar = Clean_Dataset/Corpus/Split_GramVar/Train`
  - `train_data_dir_parave = Clean_Dataset/Corpus/Split_ParaVE/Train`
- Test:
  - `test_data_dir_gramvar = Clean_Dataset/Corpus/Split_GramVar/Test`
  - `test_data_dir_parave = Clean_Dataset/Corpus/Split_ParaVE/Test`
- Các file JSON trong các thư mục này có format:
  - Mỗi phần tử là một sample, gồm:
    - `text`: câu đầy đủ.
    - `predicate`: động từ/tiêu đề nhãn.
    - `arguments`: dict `ARG-*` → text argument.

### 5. Thư viện sử dụng

- `torch`, `torch.nn.functional` (mô hình, tính gradient, phép toán tensor).
- `transformers`:
  - `AutoTokenizer`, `AutoModelForTokenClassification` (BioBERT SRL fine-tuned).
- `seqeval.metrics.f1_score`:
  - Tính F1 theo chuẩn sequence labeling (micro F1).
- `glob`, `json`, `os`:
  - Duyệt thư mục, đọc/ghi file JSON.
- `tqdm.auto`:
  - Hiển thị tiến độ vòng lặp.
- `copy`:
  - `deepcopy` mô hình để tạo phiên bản cập nhật tạm thời.


## Đầu ra

- Hai file JSON mới trong `matching_results`:

1. `faithfulness_scores_baseline1_traditional_parave_test.json`
2. `faithfulness_scores_baseline1_traditional_gramvar_test.json`

Mỗi file gồm danh sách các phần tử `result`:

- `result['test_sample']`:
  - Thông tin về mẫu test (giữ nguyên từ file influence input).
- `result['neighbors']`:
  - Danh sách các neighbor đã được cập nhật, mỗi neighbor gồm:
    - Các trường sẵn có (ví dụ `sentence_text`, `verb_name`, `influence_score_traditional`, v.v.).
    - Trường mới:
      - `DeltaF1`: chênh lệch F1 trên mẫu test sau khi cập nhật mô hình bằng neighbor:

      ```text
      DeltaF1 = F1_new(test | theta') − F1_original(test | theta)
      ```


## Quy trình

### 1. Tải mô hình gốc (theta)

- Sử dụng `AutoTokenizer` và `AutoModelForTokenClassification`:
  - Tải từ `final_model_path`.
- Thiết lập:
  - `base_model_ft.train()`:
    - Mô hình ở chế độ train để có thể tính gradient khi cần.
  - Tắt dropout và layernorm (đặt các module tương ứng ở `.eval()`):
    - Đảm bảo tính nhất quán khi so sánh F1 trước và sau update.
- Lấy:
  - `label_to_id = base_model_ft.config.label2id`
  - `id_to_label` (map ngược, dùng để decode nhãn khi tính F1).

### 2. Tải dữ liệu thô thành lookup

#### Hàm `load_raw_data_as_lookup(data_dirs)`

- Mục đích:
  - Tải **tất cả** các mẫu train/test vào một dictionary tra cứu nhanh.
- Cách làm:
  1. Duyệt tất cả file `.json` trong các thư mục `data_dirs`.
  2. Lấy `verb_name` từ tên file:
     - Ví dụ: `begin_1_train_set.json` → `begin_1`.
  3. Đọc nội dung file:
     - Mỗi `item` trong `data` được gán key:
       - `key = (item['text'], verb_name)`.
  4. Lưu vào `lookup`:
     - `lookup[(text, verb_name)] = item`.
- Áp dụng:
  - `train_lookup`:
    - Từ `train_data_dir_gramvar` và `train_data_dir_parave`.
  - `test_lookup`:
    - Từ `test_data_dir_gramvar` và `test_data_dir_parave`.

### 3. Tạo input và nhãn cho một mẫu

#### Hàm `tokenize_and_align_labels_single(item, tokenizer, label_to_id)`

- Mục đích:
  - Chuyển một `item` (JSON) thành:
    - `input_ids`, `attention_mask`, (có thể có `token_type_ids`).
    - `labels` đã căn chỉnh với tokenization (BIO tags).
- Các bước chính:
  1. Lấy:
     - `text`, `predicate`, `arguments` từ `item`.
  2. Tokenize cả câu `text`:
     - `tokenized_inputs = tokenizer(text, truncation=True, max_length=512, ...)`.
     - Lấy `tokens` và `word_ids`.
  3. Khởi tạo `label_ids = [-100] * len(word_ids)`:
     - -100 cho các token sẽ bị bỏ qua trong tính loss.
  4. Với mỗi `(arg_label, arg_text)`:
     - Bỏ qua nếu `arg_text` rỗng hoặc nhãn không bắt đầu bằng `ARG`.
     - Tokenize `arg_text`:
       - `arg_tokens = tokenizer.tokenize(arg_text, add_special_tokens=False)`.
     - Tìm vị trí dãy `arg_tokens` trong `tokens`:
       - Nếu trùng:
         - Gán nhãn `B-` ở token đầu, `I-` ở các token tiếp theo nếu tồn tại trong `label_to_id`.
  5. Tạo `final_labels`:
     - Với mỗi `(word_idx, label_id)`:
       - `word_idx is None`: gán -100.
       - `label_id != -100`: giữ nguyên.
       - Ngược lại: gán nhãn `O` (background).
  6. Đưa vào `tokenized_inputs`:
     - `labels` là tensor `[1, seq_len]`.
     - `input_ids`, `attention_mask`, `token_type_ids` (nếu có) đều thêm chiều batch.

### 4. Tính F1 cho một mẫu test

#### Hàm `get_f1_score(model, test_item, tokenizer, label_to_id, id_to_label)`

- Mục đích:
  - Tính micro F1 (theo seqeval) cho **một** mẫu test.
- Bảo toàn trạng thái:
  - Lưu lại `original_mode_is_training = model.training`.
  - Sau khi xong, khôi phục lại trạng thái ban đầu (train/eval).
- Các bước:
  1. Gọi `tokenize_and_align_labels_single` để lấy `inputs` và `labels`.
  2. Dùng `with torch.no_grad():`
     - Đặt `model.eval()`.
     - Gọi `outputs = model(**inputs)`.
     - Lấy `predictions = argmax(outputs.logits, dim=2)`.
  3. Lọc bỏ token có nhãn -100, chuyển id → label string:
     - Dùng `id_to_label`.
  4. Tính F1:
     - `seqeval_f1_score(true_labels, true_predictions, average='micro')`.
  5. Nếu có lỗi:
     - In thông báo và trả về F1 = 0.0.
  6. Khôi phục lại trạng thái của model:
     - Nếu trước đó là `.train()`, đặt lại `.train()`.

### 5. Một bước gradient descent trên mẫu train

#### Hàm `perform_gradient_step(base_model, train_item, tokenizer, label_to_id, eta)`

- Mục đích:
  - Tạo mô hình mới `theta'` bằng **một bước SGD** trên **một** mẫu train.
- Các bước:
  1. `temp_model = copy.deepcopy(base_model)`:
     - Đảm bảo base model không bị thay đổi.
  2. Đặt `temp_model.train()`.
  3. Tắt dropout và layernorm:
     - Cho tất cả module `Dropout` và `LayerNorm`: `.eval()`.
  4. Chuẩn bị `inputs` và `labels` từ `train_item`.
  5. Tính loss:
     - `outputs = temp_model(**inputs, labels=labels)`.
     - `loss = outputs.loss`.
  6. Khởi tạo optimizer SGD:
     - `optimizer = torch.optim.SGD(temp_model.parameters(), lr=eta)`.
  7. `optimizer.zero_grad()`, `loss.backward()`, `optimizer.step()`.
  8. Đặt `temp_model.eval()` sau khi cập nhật.
  9. Trả về `temp_model` (mô hình `theta'`).

### 6. Vòng lặp chính: tính DeltaF1 cho Baseline 1

Trong `if __name__ == "__main__":`:

1. Tải mô hình fine-tuned:
   - `base_model_ft` + tokenizer + mapping nhãn.
2. Tải dữ liệu thô:
   - `train_lookup`, `test_lookup`.

3. Với mỗi bộ dữ liệu trong `datasets_to_run_faithfulness`:

   - Đọc file influence input (`knn_file`):
     - `influence_results_baseline1_traditional_*.json`.
   - Tạo list `faithfulness_results = []`.

   - Với mỗi `result` trong `knn_results`:
     1. Lấy `test_sample_info = result['test_sample']`.
     2. Tạo `test_key = (test_sample_info['sentence_text'], test_sample_info['verb_name'])`.
     3. Lấy `test_item = test_lookup.get(test_key)`:
        - Nếu không có: bỏ qua.
     4. Tính `F1_original` từ base model trên `test_item`.
     5. Khởi tạo `processed_neighbors = []`.
     6. Duyệt các neighbor:
        - `for neighbor in result['neighbors'][:K_NEIGHBORS_TO_EVALUATE]:`
          - Lấy `train_key = (neighbor['sentence_text'], neighbor['verb_name'])`.
          - Lấy `train_item` từ `train_lookup`.
          - Nếu không có: bỏ qua.
          - Gọi `perform_gradient_step` để tạo `temp_model = theta'`.
          - Tính `F1_new` = F1 của `temp_model` trên `test_item`.
          - Tính `DeltaF1 = F1_new - F1_original`.
          - Gán `neighbor['DeltaF1'] = delta_f1`.
          - Thêm `neighbor` vào `processed_neighbors`.
          - `del temp_model` để giải phóng bộ nhớ.
     7. Gán lại:
        - `result['neighbors'] = processed_neighbors`.
     8. Thêm `result` vào `faithfulness_results`.

   - Sau khi duyệt hết:
     - Ghi `faithfulness_results` ra `faithfulness_file` tương ứng.

4. In thông báo hoàn tất:
   - `--- QUÁ TRÌNH ĐÁNH GIÁ FAITHFULNESS (BASELINE 1) ĐÃ HOÀN TẤT! ---`

In [ ]:
# --- CÀI ĐẶT THƯ VIỆN CẦN THIẾT ---
!pip install seqeval

# Import các thư viện cần thiết
import torch
import os
import json
import glob
from tqdm.auto import tqdm
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForTokenClassification
import copy # Cần thiết để deepcopy mô hình
from seqeval.metrics import f1_score as seqeval_f1_score # Để tính F1
import re

# --- 1. CẤU HÌNH ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# --- CẤU HÌNH THÍ NGHIỆM ---
ETA_LEARNING_RATE = 1e-1 # 0.1
K_NEIGHBORS_TO_EVALUATE = 1 # Chỉ đánh giá 3 láng giềng gần nhất

# --- 2. THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'
output_dir = os.path.join(drive_base_path, 'matching_results')

# Đường dẫn đến mô hình fine-tuned
final_model_path = os.path.join(drive_base_path, 'Finetuned_Models/biobert-srl-best-model')

# *** CẬP NHẬT: Danh sách các bộ dữ liệu đã có kết quả influence (BASELINE 1) ***
datasets_to_run_faithfulness = [
    (
        "ParaVE Test (Baseline 1)",
        os.path.join(output_dir, 'influence_results_baseline1_traditional_parave_test.json'), # File Influence đầu vào MỚI
        os.path.join(output_dir, 'faithfulness_scores_baseline1_traditional_parave_test.json') # File DeltaF1 đầu ra MỚI
    ),
    (
        "GramVar Test (Baseline 1)",
        os.path.join(output_dir, 'influence_results_baseline1_traditional_gramvar_test.json'), # File Influence đầu vào MỚI
        os.path.join(output_dir, 'faithfulness_scores_baseline1_traditional_gramvar_test.json') # File DeltaF1 đầu ra MỚI
    )
]

# Đường dẫn đến các file dữ liệu GỐC (để lấy thông tin đầy đủ)
train_data_dir_gramvar = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Train')
train_data_dir_parave = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Train')
test_data_dir_gramvar = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test')
test_data_dir_parave = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test')

# --- 3. CÁC HÀM XỬ LÝ CỐT LÕI ---

def load_raw_data_as_lookup(data_dirs):
    """Tải tất cả dữ liệu thô vào dict, dùng (text, verb_name) làm key."""
    print("Đang tải dữ liệu thô vào bộ nhớ đệm...")
    lookup = {}
    if not isinstance(data_dirs, list): data_dirs = [data_dirs]

    for data_dir in data_dirs:
        json_files = glob.glob(os.path.join(data_dir, '*.json'))
        for json_file in tqdm(json_files, desc=f"Tải {os.path.basename(data_dir)}", leave=False):
            try:
                base_name = os.path.basename(json_file)
                verb_name = "_".join(base_name.replace('_train_set.json', '').replace('_test_set.json', '').split('_'))

                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for item in data:
                        key = (item['text'], verb_name) # Dùng (text, verb_name) làm key
                        lookup[key] = item
            except Exception as e:
                print(f"Lỗi khi đọc file {json_file}: {e}")
    print(f" -> Đã tải {len(lookup)} mẫu.")
    return lookup

def tokenize_and_align_labels_single(item, tokenizer, label_to_id):
    """Tạo input_ids và labels cho MỘT mẫu duy nhất."""
    text, predicate, arguments = item['text'], item['predicate'], item['arguments']
    tokenized_inputs = tokenizer(text, truncation=True, max_length=512, is_split_into_words=False)
    word_ids = tokenized_inputs.word_ids(batch_index=0)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs.input_ids)
    label_ids = [-100] * len(word_ids)

    for arg_label, arg_text in arguments.items():
        if not arg_text or not arg_label.startswith('ARG'): continue
        arg_tokens = tokenizer.tokenize(arg_text, add_special_tokens=False)
        if not arg_tokens: continue
        for token_start_idx in range(len(tokens) - len(arg_tokens) + 1):
            if tokens[token_start_idx : token_start_idx + len(arg_tokens)] == arg_tokens:
                b_label_name = f"B-{predicate}-{arg_label}"
                if b_label_name in label_to_id: label_ids[token_start_idx] = label_to_id[b_label_name]
                i_label_name = f"I-{predicate}-{arg_label}"
                if i_label_name in label_to_id:
                    for j in range(1, len(arg_tokens)): label_ids[token_start_idx + j] = label_to_id[i_label_name]
                break
    final_labels = []
    for word_idx, label_id in zip(word_ids, label_ids):
        if word_idx is None: final_labels.append(-100)
        elif label_id != -100: final_labels.append(label_id)
        else: final_labels.append(label_to_id.get('O', 0))
    tokenized_inputs['labels'] = torch.tensor(final_labels).unsqueeze(0)
    tokenized_inputs['input_ids'] = torch.tensor(tokenized_inputs.input_ids).unsqueeze(0)
    tokenized_inputs['attention_mask'] = torch.tensor(tokenized_inputs.attention_mask).unsqueeze(0)
    if 'token_type_ids' in tokenized_inputs:
       tokenized_inputs['token_type_ids'] = torch.tensor(tokenized_inputs.token_type_ids).unsqueeze(0)
    return tokenized_inputs

def get_f1_score(model, test_item, tokenizer, label_to_id, id_to_label):
    """
    Tính F1 score (micro) cho một mẫu test duy nhất.
    *** Đã sửa lỗi: Bảo toàn trạng thái của model (train/eval) ***
    """
    original_mode_is_training = model.training # Lưu trạng thái

    try:
        inputs = tokenize_and_align_labels_single(test_item, tokenizer, label_to_id)
        labels = inputs.pop('labels').to(device)
        inputs = {k: v.to(device) for k, v in inputs.items()}
    except Exception:
        if original_mode_is_training: model.train() # Khôi phục trạng thái
        return 0.0

    f1 = 0.0
    try:
        with torch.no_grad():
            model.eval() # Chuyển sang eval để dự đoán
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=2)

        true_labels, true_predictions = [], []
        labels_cpu, predictions_cpu = labels[0].cpu().numpy(), predictions[0].cpu().numpy()

        current_label_list, current_pred_list = [], []
        for (pred, label) in zip(predictions_cpu, labels_cpu):
            if label != -100:
                current_label_list.append(id_to_label.get(label, 'O'))
                current_pred_list.append(id_to_label.get(pred, 'O'))
        true_labels.append(current_label_list)
        true_predictions.append(current_pred_list)

        f1 = seqeval_f1_score(true_labels, true_predictions, average='micro')
    except Exception as e:
        print(f"Lỗi khi tính F1: {e}")
        f1 = 0.0 # Gán F1 = 0 nếu có lỗi

    # Khôi phục trạng thái
    if original_mode_is_training:
        model.train()

    return f1

def perform_gradient_step(base_model, train_item, tokenizer, label_to_id, eta):
    """Tạo một mô hình mới (theta') bằng cách thực hiện 1 bước gradient descent."""
    temp_model = copy.deepcopy(base_model)
    temp_model.train() # Đặt ở .train() để tính grad

    # Tắt Dropout và LayerNorm bằng tay
    for module in temp_model.modules():
        if isinstance(module, (torch.nn.Dropout, torch.nn.LayerNorm)):
            module.eval()

    try:
        inputs = tokenize_and_align_labels_single(train_item, tokenizer, label_to_id)
        labels = inputs.pop('labels').to(device)
        inputs = {k: v.to(device) for k, v in inputs.items()}
    except Exception: return None

    with torch.enable_grad():
        outputs = temp_model(**inputs, labels=labels)
        loss = outputs.loss

    if loss is None: return None

    optimizer = torch.optim.SGD(temp_model.parameters(), lr=eta)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    temp_model.eval() # Chuyển về chế độ eval sau khi xong
    return temp_model # Trả về theta'

# --- 4. THỰC THI CHƯƠNG TRÌNH ---
if __name__ == "__main__":

    # --- Bước 1: Tải mô hình gốc (theta) ---
    print("--- Bước 1: Tải mô hình Fine-tuned (theta) ---")
    try:
        tokenizer = AutoTokenizer.from_pretrained(final_model_path)
        base_model_ft = AutoModelForTokenClassification.from_pretrained(final_model_path).to(device)

        # Đặt ở .train() và tắt dropout/layernorm
        base_model_ft.train()
        for module in base_model_ft.modules():
            if isinstance(module, (torch.nn.Dropout, torch.nn.LayerNorm)):
                module.eval()

        label_to_id = base_model_ft.config.label2id
        id_to_label = {v: k for k, v in label_to_id.items()}
        print(" -> Tải mô hình và tokenizer thành công.")
    except Exception as e:
        print(f"LỖI: Không thể tải mô hình: {e}")
        exit()

    # --- Bước 2: Tải dữ liệu thô (Tải 1 lần) ---
    print("\n--- Bước 2: Tải dữ liệu thô ---")
    train_lookup = load_raw_data_as_lookup([train_data_dir_gramvar, train_data_dir_parave])
    test_lookup = load_raw_data_as_lookup([test_data_dir_gramvar, test_data_dir_parave])

    # --- Bước 3: Vòng lặp xử lý từng bộ dữ liệu ---
    print("\n--- Bước 3: Bắt đầu tính toán Faithfulness (BASELINE 1 - IF TRUYỀN THỐNG) ---")

    for name, knn_file, faithfulness_file in datasets_to_run_faithfulness:

        print(f"\n=================================================")
        print(f"BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: {name}")
        print(f"=================================================")

        try:
            with open(knn_file, 'r', encoding='utf-8') as f:
                knn_results = json.load(f)
            print(f" -> Tải thành công {len(knn_results)} kết quả kNN từ {knn_file}.")
        except FileNotFoundError:
            print(f"LỖI: Không tìm thấy file kNN: {knn_file}. Bỏ qua bộ này.")
            continue

        faithfulness_results = []

        for result in tqdm(knn_results, desc=f"Tính DeltaF1 (Baseline 1) cho {name}"):
            test_sample_info = result['test_sample']
            test_key = (test_sample_info['sentence_text'], test_sample_info['verb_name'])
            test_item = test_lookup.get(test_key)
            if not test_item: continue

            # 1. Tính F1 gốc (chỉ 1 lần cho mỗi mẫu test)
            F1_original = get_f1_score(base_model_ft, test_item, tokenizer, label_to_id, id_to_label)

            processed_neighbors = []
            for neighbor in result['neighbors'][:K_NEIGHBORS_TO_EVALUATE]:
                train_key = (neighbor['sentence_text'], neighbor['verb_name'])
                train_item = train_lookup.get(train_key)
                if not train_item: continue

                # 2. Tạo mô hình mới theta'
                temp_model = perform_gradient_step(base_model_ft, train_item, tokenizer, label_to_id, ETA_LEARNING_RATE)
                if temp_model is None:
                    continue

                # 3. Tính F1 mới
                F1_new = get_f1_score(temp_model, test_item, tokenizer, label_to_id, id_to_label)

                # 4. Tính DeltaF1
                delta_f1 = F1_new - F1_original
                neighbor['DeltaF1'] = delta_f1

                # *** THAY ĐỔI: Chỉ cần đảm bảo key 'influence_score_traditional' được giữ lại ***
                # Script này không ghi đè, nó chỉ thêm 'DeltaF1', nên score gốc vẫn được giữ.

                processed_neighbors.append(neighbor)

                del temp_model

            result['neighbors'] = processed_neighbors
            faithfulness_results.append(result)

        # 3c. Lưu kết quả
        print(f"\n--- Lưu kết quả cho {name} ---")
        with open(faithfulness_file, 'w', encoding='utf-8') as f:
            json.dump(faithfulness_results, f, indent=4, ensure_ascii=False)
        print(f" -> Đã lưu thành công {len(faithfulness_results)} kết quả vào: {faithfulness_file}")

    print("\n--- QUÁ TRÌNH ĐÁNH GIÁ FAITHFULNESS (BASELINE 1) ĐÃ HOÀN TẤT! ---")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=3c405afd411a510e5d007b60f4e1e71cf82e202b728ea2ee78ea5d3894aab513
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval
Sử dụng thiết bị: cuda
--- Bước 1: Tải mô hình Fine-tuned (theta) ---
 -> Tải mô hình và tokenizer thành công.

--- Bước 2: Tải dữ liệu thô ---
Đang tải dữ liệu thô vào bộ nhớ đệm...


Tải Train:   0%|          | 0/35 [00:00<?, ?it/s]

Tải Train:   0%|          | 0/35 [00:00<?, ?it/s]

 -> Đã tải 7327 mẫu.
Đang tải dữ liệu thô vào bộ nhớ đệm...


Tải Test:   0%|          | 0/35 [00:00<?, ?it/s]

...

 -> Đã tải 1185 mẫu.

--- Bước 3: Bắt đầu tính toán Faithfulness (BASELINE 1 - IF TRUYỀN THỐNG) ---

BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: ParaVE Test (Baseline 1)
 -> Tải thành công 699 kết quả kNN từ /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/matching_results/influence_results_baseline1_traditional_parave_test.json.


Tính DeltaF1 (Baseline 1) cho ParaVE Test (Baseline 1):   0%|          | 0/699 [00:00<?, ?it/s]


--- Lưu kết quả cho ParaVE Test (Baseline 1) ---
 -> Đã lưu thành công 699 kết quả vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/matching_results/faithfulness_scores_baseline1_traditional_parave_test.json

BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: GramVar Test (Baseline 1)
 -> Tải thành công 1711 kết quả kNN từ /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/matching_results/influence_results_baseline1_traditional_gramvar_test.json.


Tính DeltaF1 (Baseline 1) cho GramVar Test (Baseline 1):   0%|          | 0/1711 [00:00<?, ?it/s]


--- Lưu kết quả cho GramVar Test (Baseline 1) ---
 -> Đã lưu thành công 1711 kết quả vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/matching_results/faithfulness_scores_baseline1_traditional_gramvar_test.json

--- QUÁ TRÌNH ĐÁNH GIÁ FAITHFULNESS (BASELINE 1) ĐÃ HOÀN TẤT! ---
